# Run this to update the img_path_match column for tbl_webpage_images

In [ ]:
number_of_rows_to_run = 70000
start_id = 1
end_id = start_id+number_of_rows_to_run

import mysql.connector as mariadb
mariadb_connection = mariadb.connect(option_files='C:\\ProgramData\\MySQL\\MySQL Server 8.0\\webpage_images.cnf', option_groups="webpage_images")
cursor = mariadb_connection.cursor(buffered=True)
cursor_update = mariadb_connection.cursor(buffered=True)
cursor.execute("SELECT id, folder, img_src FROM webpage_images.tbl_webpage_images WHERE id>=%s AND id<%s", (start_id, end_id))
for id, folder, img_src in cursor:
    img_path_match = folder.replace("/pompeiiinpictures", "")+"/"+img_src.replace("%20"," ")
    print(img_path_match)
    update_image = ("UPDATE tbl_webpage_images SET `img_path_match` = %s WHERE id=%s;")
    #print(update_image)
    cursor_update.execute(update_image, (img_path_match, id))
    mariadb_connection.commit()
    
mariadb_connection.close()    


# Run this to update the img_path_match column for tbl_box_images

In [ ]:
number_of_rows_to_run = 290000
start_id = 1
end_id = start_id+number_of_rows_to_run

import mysql.connector as mariadb
mariadb_connection = mariadb.connect(option_files='C:\\ProgramData\\MySQL\\MySQL Server 8.0\\webpage_images.cnf', option_groups="webpage_images")
cursor = mariadb_connection.cursor(buffered=True)
cursor_update = mariadb_connection.cursor(buffered=True)
cursor.execute("SELECT id, img_src FROM webpage_images.tbl_box_images WHERE id>=%s AND id<%s", (start_id, end_id))
for id, img_src in cursor:
    #print(img_src)
    #print(img_src[:7])
    if(img_src[:7]=="/pinp5/") and (img_src.lower().find(".xml")<0) and (img_src.lower().find(".htm")<0) and (img_src.lower().find(".thmx")<0):
        img_path_match = img_src.replace("/pinp5/","")
        img_path_match = img_path_match[img_path_match.find("/"):]
        print(img_path_match)
        update_image = ("UPDATE tbl_box_images SET `img_path_match` = %s WHERE id=%s;")
        #print(update_image)
        cursor_update.execute(update_image, (img_path_match, id))
        mariadb_connection.commit()
    
mariadb_connection.close()    


# Run this to get a list of all image tags to be insert into a table

In [ ]:
tags = {}
def azure_img_tags_parse(tag_text):
    # Tags are formatted like: [outdoor] [tree] [ruin] [ruins] [stone] [wall] [archaeology] [ancient] [sky] [stone wall] [plant] [concrete] [cement]
    #print(tag_text)
    if (tag_text is None):
        return

    if (tag_text.strip() == ""):
        return
    
    # if there is no [ assume there are no more tags
    if(tag_text.find("[")<0):
        return
    else:
        #print(tag_text)
        lb = tag_text.find("[")
        rb = tag_text.find("]")
        tag = tag_text[lb+1:rb].lower()
        #print(tag)
        tags[tag] = tag
        azure_img_tags_parse(tag_text[tag_text.find("]")+1:])
                   
number_of_rows_to_run = 70000
start_id = 1
end_id = start_id+number_of_rows_to_run

import mysql.connector as mariadb
mariadb_connection = mariadb.connect(option_files='C:\\ProgramData\\MySQL\\MySQL Server 8.0\\webpage_images.cnf', option_groups="webpage_images")
cursor = mariadb_connection.cursor(buffered=True)
cursor.execute("SELECT azure_img_tags FROM webpage_images.tbl_webpage_images WHERE id>=%s AND id<%s", (start_id, end_id))
for azure_img_tags in cursor:
    
    #print(azure_img_tags[0])
    azure_img_tags_parse(azure_img_tags[0])
mariadb_connection.close()
                   
for t, t1 in sorted(tags.items()):    
    print (t)

In [ ]:
#get max size for length of field
max_len = 0
for t, t1 in sorted(tags.items()):    
    print (t, len(t))
    max_len = max(max_len,len(t))    
print(max_len)

In [ ]:
# insert tags into table
import mysql.connector as mariadb
mariadb_connection = mariadb.connect(option_files='C:\\ProgramData\\MySQL\\MySQL Server 8.0\\webpage_images.cnf', option_groups="webpage_images")
cursor_insert = mariadb_connection.cursor(buffered=True)

for t, t1 in sorted(tags.items()):    
    
    t=t.replace("'","''")
    print (t)
    cursor_insert.execute("INSERT INTO tbl_azure_img_tags (azure_img_tag) VALUES ('"+t+"')")    
    mariadb_connection.commit()
    
mariadb_connection.close()



# Update image descriptions

In [ ]:
number_of_rows_to_run = 70000
start_id = 1
end_id = start_id+number_of_rows_to_run

import mysql.connector as mariadb
mariadb_connection = mariadb.connect(option_files='C:\\ProgramData\\MySQL\\MySQL Server 8.0\\webpage_images.cnf', option_groups="webpage_images")
cursor = mariadb_connection.cursor(buffered=True)
cursor_aid = mariadb_connection.cursor(buffered=True)
cursor_update_wi = mariadb_connection.cursor(buffered=True)

#Do image descriptions first
cursor.execute("SELECT id as wi_id, azure_img_desc FROM webpage_images.tbl_webpage_images WHERE id>=%s AND id<%s", (start_id, end_id))
for wi_id, azure_img_desc in cursor:
        
    #print(azure_img_desc)
    if(not azure_img_desc is None):
        cursor_aid.execute("SELECT id as aid_id FROM webpage_images.tbl_azure_img_desc WHERE azure_img_desc='"+azure_img_desc+"'")
        for aid_id in cursor_aid:
            print(aid_id[0])
            cursor_update_wi.execute("UPDATE tbl_webpage_images SET `azure_img_desc_id` = %s WHERE id=%s;",(aid_id[0],wi_id))            
            mariadb_connection.commit()
mariadb_connection.close()

# Update image tags

In [ ]:
tags = {}
def azure_img_tags_parse_insert_into_table(tag_text, wi_id, cursor_ait, cursor_ait_x, cursor_update_ait_x):
    # Tags are formatted like: [outdoor] [tree] [ruin] [ruins] [stone] [wall] [archaeology] [ancient] [sky] [stone wall] [plant] [concrete] [cement]
    #print(tag_text)
    if (tag_text is None):
        return

    if (tag_text.strip() == ""):
        return
    
    # if there is no [ assume there are no more tags
    if(tag_text.find("[")<0):
        return
    else:
        #print(tag_text)
        lb = tag_text.find("[")
        rb = tag_text.find("]")
        tag = tag_text[lb+1:rb].lower()
        #print(tag)
        #tags[tag] = tag
        
        #We have a tag.  Get its ID and then insert it into the cross reference table tbl_azure_img_tags_x with the ID of the image
        tag=tag.replace("'","''")       
        cursor_ait.execute("SELECT id as ait_id FROM webpage_images.tbl_azure_img_tags WHERE azure_img_tag='"+tag+"'")        
        for ait_id in cursor_ait:
            #print(ait_id)
            cursor_ait_x.execute("SELECT count(*) as cn FROM webpage_images.tbl_azure_img_tags_x WHERE ait_id=%s AND wi_id=%s",(ait_id[0],wi_id))
            for cn in cursor_ait_x:
                #print("count " + str(cn[0]))
                if(cn[0]==0):
                    cursor_update_ait_x.execute("INSERT INTO tbl_azure_img_tags_x (ait_id,wi_id) VALUES (%s, %s)",(ait_id[0],wi_id))
                    mariadb_connection.commit()
                #else:
                    #print(str(ait_id[0])+" "+str(wi_id)+" are already in the table.")
                    
        azure_img_tags_parse_insert_into_table(tag_text[tag_text.find("]")+1:], wi_id, cursor_ait, cursor_ait_x, cursor_update_ait_x)
                   
number_of_rows_to_run = 70000
start_id = 1
end_id = start_id+number_of_rows_to_run

import mysql.connector as mariadb
mariadb_connection = mariadb.connect(option_files='C:\\ProgramData\\MySQL\\MySQL Server 8.0\\webpage_images.cnf', option_groups="webpage_images")
cursor = mariadb_connection.cursor(buffered=True)
cursor_ait = mariadb_connection.cursor(buffered=True)
cursor_ait_x = mariadb_connection.cursor(buffered=True)
cursor_update_ait_x = mariadb_connection.cursor(buffered=True)

#Do image descriptions first
cursor.execute("SELECT id as wi_id, azure_img_desc, azure_img_tags FROM webpage_images.tbl_webpage_images WHERE id>=%s AND id<%s", (start_id, end_id))
for wi_id, azure_img_desc, azure_img_tags in cursor:
    
    print(wi_id)
    if(not azure_img_tags is None):
        #print(azure_img_tags)
        azure_img_tags_parse_insert_into_table(azure_img_tags, wi_id, cursor_ait, cursor_ait_x, cursor_update_ait_x)
        #print(azure_img_tags)
mariadb_connection.close()

In [32]:
mariadb_connection.close()